### Quick intro to Large Language Models

Tìm hiểu cách LLM học các phân phối token và dự đoán token tiếp theo, cho phép nó tạo ra đoạn text giống người viết.
Thông thường mỗi mô hình sẽ giới hạn số lượng token, nếu câu văn quá dài sẽ dấn đến việc bị mất token.
Hướng xử lý: bạn có thể cắt mã và đẩy lần lượt.

In [4]:
from utils import setup_openai_key
from langchain.llms import OpenAI

In [5]:
# Save OpenAI key in the environment
setup_openai_key()

In [16]:
llm = OpenAI(model_name="text-davinci-003")

#### Tokens Distributions and Predicting the Next Token

In [8]:
text = "What would be a good company name for a company that makes colorful socks?"

print(llm(text))



Happy Sockz Co.


#### Tracking Token Usage

In [16]:
from langchain.callbacks import get_openai_callback

In [15]:
llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)

In [18]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 44
	Prompt Tokens: 4
	Completion Tokens: 40
Successful Requests: 1
Total Cost (USD): $0.00088


In [19]:
# Callback giúp track đc tokens đã dùng, số lượng request và cost

#### Few-shot learning

Học khái quát hóa từ các ví dụ hạn chế.

In [20]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

In [21]:
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

In [22]:
# Tạo template
example_template = """
User: {query}
AI: {answer}
"""

In [23]:
# Tạo prompt dự trên template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

In [24]:
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""

In [25]:
suffix = """
User: {query}
AI: """

In [26]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

In [42]:
# load the model
chat = ChatOpenAI( temperature=0.0)

In [43]:
chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

'To find the perfect balance between pizza and ice cream.'

#### Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering

In [44]:
#### Creating a Question-Answering Prompt Template

In [3]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [4]:
from langchain import HuggingFaceHub, LLMChain

In [5]:
# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

E:\Hoc\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\Hoc\LangChain\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

In [7]:
# ask the user question about the capital of France
print(llm_chain.run(question))

paris


In [8]:
#### Asking Multiple Questions

In [9]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]

In [10]:
res = llm_chain.generate(qa)

In [11]:
print( res )

generations=[[Generation(text='paris', generation_info=None)], [Generation(text='giraffe', generation_info=None)], [Generation(text='nitrogen', generation_info=None)], [Generation(text='yellow', generation_info=None)]] llm_output=None run=RunInfo(run_id=UUID('67b33511-8452-4555-a645-17ee7ffb6dc4'))


#### Chúng ta có thể sửa đổi mẫu lời nhắc của mình để bao gồm nhiều câu hỏi nhằm triển khai phương pháp thứ hai. Mô hình ngôn ngữ sẽ hiểu rằng chúng ta có nhiều câu hỏi và trả lời chúng một cách tuần tự.

In [12]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

In [13]:
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])


In [17]:
llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

In [18]:
qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

'Paris\nBlue whale\nNitrogen\nYellow'

## Text Summarization

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [20]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [21]:
summarization_template = "Summarize the following text to one sentence: {text}"

In [22]:
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)

In [23]:
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [24]:
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."


In [25]:
summarized_text = summarization_chain.predict(text=text)

In [26]:
print(summarized_text)

LangChain offers various modules for building language model applications, allowing users to combine them for more complex applications or use them individually for simpler ones, with the basic building block being calling an LLM on input, as demonstrated in the example of creating a company name based on its product.


## Text Translation

In [27]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"


In [28]:
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)


In [29]:
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [30]:
source_language = "English"
target_language = "French"
text = "My name is Gioi"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

In [31]:
print(translated_text)

Je m'appelle Gioi


# Building Applications Powered by LLMs with LangChain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [5]:
# ChatPromptTemplate: tạo cấu trúc cuộc trò chuyện, quản lý luồng và nội dung của đoạn hội thoại.
# SystemMessagePromptTemplate cung cấp hướng dẫn, ngữ cảnh hoặc dữ liệu ban đầu cho mô hình AI.
# HumanMessagePromptTemplate là các tin nhắn từ người dùng mà mô hình AI phản hồi.

In [6]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [7]:
template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

In [8]:
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [9]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


In [10]:
response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())


In [11]:
print(response.content)

"Inception" is a science fiction action film directed by Christopher Nolan. It was released in 2010 and stars Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy, and Marion Cotillard. The film follows a professional thief who steals information by infiltrating the subconscious of his targets through their dreams. 

The story revolves around Dom Cobb (played by DiCaprio), who is offered a chance to have his criminal history erased in exchange for performing the act of "inception" - planting an idea in someone's mind rather than stealing it. As Cobb and his team navigate through various dream levels, they encounter challenges and face the consequences of manipulating the subconscious.

"Inception" received critical acclaim for its originality, visual effects, and complex narrative. It was praised for its thought-provoking themes and exploration of the nature of reality and dreams. The film was a commercial success, grossing over $828 million worldwide.

It won four Academy Aw

### Summarization chain example

In [1]:
# Cần cài được pypdf 3.10.0

In [6]:
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

In [7]:
llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [8]:
summarize_chain = load_summarize_chain(llm)

In [9]:
document_loader = PyPDFLoader(file_path=r"E:\CV_matching\CV_Dataset\kaggle_resume\ADVOCATE\10344379.pdf")
document = document_loader.load()

In [10]:
# Summarize the document
summary = summarize_chain(document)
print(summary['output_text'])

 Administrative support professional with experience in a fast-paced environment, requiring strong organizational, technical, and interpersonal skills. Experienced in customer service, logistics/distribution management, medical device repair, production/operations supervision, shipping/receiving, and excellent written/verbal communication. Has a post-secondary training certificate in Logistics and Supply Chain Management and a diploma from Concorde Career Institution. Also has 84-92 US Army experience as a Communications Specialist.


### QA chain example

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [12]:
prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])


In [13]:
llm = OpenAI(model_name="text-davinci-003", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

In [14]:
chain.run("what is the meaning of life?")

' The meaning of life is subjective and can vary from person to person. For some, it may be to find happiness and fulfillment, while for others it may be to make a difference in the world. Ultimately, the meaning of life is up to each individual to decide.'

In [15]:
chain.run("What is your name?")

' My name is [your name].'